In [17]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

file_path = "Processed_Data/Mapped_Dataset.csv"

df = dk.read_csv(file_path)

In [ ]:
#Global var
batch_size = 128
ratio_test_all = 0.2

from dask_ml.model_selection import train_test_split
# chia train test ratio 0.8:0.2 & random
train_df, test_df = train_test_split(df, test_size=ratio_test_all, random_state=42)


# load từng batch
def dask_to_tf_dataset(dask_df, batch_size=128, num_classes=10):
    # def generator():
    for batch in dask_df.to_delayed():
        batch = batch.compute()  # Chỉ load batch nhỏ vào RAM
        X = batch.drop(columns=["label"]).values
        y = batch["label"].values
        y_onehot = to_categorical(y, num_classes=num_classes)
        yield X, y_onehot

    # output_signature = (
    #     tf.TensorSpec(shape=(None, 46), dtype=tf.float32),
    #     tf.TensorSpec(shape=(None,10), dtype=tf.int32),
    # )
    # return tf.data.Dataset.from_generator(generator, output_signature=output_signature).batch(batch_size)

c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask_ml\model_selection\_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [21]:
#load yield
train_ds = dask_to_tf_dataset(train_df, batch_size=batch_size, num_classes=10)
test_ds = dask_to_tf_dataset(test_df, batch_size=batch_size, num_classes=10)


from tensorflow import keras
# Định nghĩa mô hình CNN
# VGG, ...
# Conv2D, tabular, ...
# HE, tính tương thích của HE với CNN
# Tính chất data in, out; Học tăng cường

model = keras.Sequential([
    layers.Input(shape=(46,1)),
    layers.Conv1D(filters=32, kernel_size=3, padding="same", activation="relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_ds, epochs=10, batch_size=batch_size, steps_per_epoch=100, verbose=1)

# Lưu mô hình
model.save("cnn_model_batch128.h5")

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv1d_5" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, None, 46)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, 46), dtype=float32)
  • training=True
  • mask=None